## Лабораторная 6: Анализ текстов. SVM

В этом задании мы должны научить машину понимать тексты. В файле recs.txt находятся отзывы о разных фильмах. Наша модель машинного обучения должна проанализировать отзыв и ответить на простой вопрос: понравился ли пользователю просмотренный фильм.

1. Подключите необходимые для работы пакеты: pandas as pd, numpy as np, nltk, re, все функции из sklearn

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
import sklearn

2. Увеличьте ширину выводимой информации в DataFrame (на это понадобится для показа текстов отзывав):
pd.set_option('max_colwidth',600)

In [2]:
pd.set_option('max_colwidth',600)

3. Прочтите данные из файла recs.txt, в качестве разделителя (параметр sep) там выступает строка "\\-\\-\\|\\|\\|\\-\\-", кодировка (параметр encoding) - 'utf-8'.
Выведите три первых строки таблицы. Какое отношение (Sentiment) у авторов отзывов к просмотренным фильмам?

In [3]:
data = pd.read_csv('recs.txt', sep='\-\-\|\|\|\-\-', encoding = 'utf-8', engine='python')
data.head(3)

,Sentiment,URL,Text
0,-1,kinopoisk.ru/user/72383/comment/846161/,"На самых первых титрах у меня появилось абсолютно твёрдое и непоколебимое ощущение, что я смотрю совершено детский фильм, в духе «Детей шпионов», иначе как объяснить наличие в начальных титрах техно-музыки и огромнейшего, просто колоссальнейшего наличия компьютерной графики, уровня приставок фирмы Sega. Если кто из вас по детству удосужился сыграть в видео-игру по мотивам мультфильма «Король-лев», адаптированную специально для этого платформера, тот прекрасно меня поймёт. Для полноты картины не хватает только мангуста Димона, который выдал бы в камеру памятное «It starts!». Хотя в данном ф..."
1,1,kinopoisk.ru/user/697175/comment/805374/,"Вы знаете, неожиданно хороший фильм. Я не большой любитель фильмов такого жанра, но этот мне очень понравился, совсем не ожидала. Придраться, как бы сильно ни хотелось это сделать, не к чему: Сюжет необычный, непривычный, оригинальный и оторваться от просмотра практически невозможно. Весь фильм проходит в движении: вечно что-то случается, открываются какие-то тайны, всевозможные неожиданные повороты и напряженные моменты. Как я уже сказала, если начнете смотреть этот фильм, остановиться уже не сможете. Я, к примеру, просто не смогла оторваться, практически прилипла к экрану. Персонажи - за..."
2,1,kinopoisk.ru/user/1335875/comment/2079675/,"Да, этот фильм такой, что всё, что там покажут, можно было угадать заочно. Достаточно было посмотреть на постер и прочесть название. Однако решил ввязаться в просмотр. В принципе, могу сказать, что не жалею времени, потраченного на это кино. Хоть это и не выдающаяся картина, было увлекательно посмотреть на приятных глазу актёров, а временами даже смешно. Это мелодрама с элементами комедии. Или любовная комедия. Или комедия с девичьим уклоном. Однако, думаю, и парням будет посмотреть что. В дебюте так вообще главные герои занимаются сексом везде и в каждом месте, словно слетевшие с катушек...."


In [4]:
data['Sentiment'].unique()
#отрицательное и положительное впечатление?

array([-1,  1], dtype=int64)

4. Чтобы преобразовать тексты в наборы признаков методом bag-of-words мы должны сначала профильтровать текст, оставив только значащие слова. Скачайте из корпуса набор незначащих слов для фильтрации командой nltk.download()  (в появившемся окне вкладка Corpora, пункт stopwords). 

In [5]:
nltk.download('stopwords')

5. Импортируйте список незначащих слов командами 

```python
from nltk.corpus import stopwords
stopwords = set(stopwords.words("russian"))
```
и выведите его на экран.

In [6]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words("russian"))
stopwords

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

6. Определите функцию фильтрации текста:

```python
def review_to_words( review_text ):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()                             
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    # 4. Формируем текст, объединяя слова через пробел
    return( " ".join( meaningful_words ))
```

In [7]:
def review_to_words( review_text ):
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё','Е')
    
    # 1. Удаляем все, кроме букв
    letters_only = re.sub("[^а-яА-Я]", " ", review_text) 
    
    # 2. Делаем все буквы строчными и создаем массив слов
    words = letters_only.lower().split()
    
    # 3. Удаляем незначащие слова
    meaningful_words = [w for w in words if not w in stopwords]
    
    # 4. Формируем текст, объединяя слова через пробел
    return( " ".join( meaningful_words ))

7. Скопируйте базу данных отзывов с помощью метода copy() в новую переменную и примените к отзывам функцию фильтрации: .apply(lambda s: review_to_words(s)).
Выведите на экран первые три строчки преобразованной базы данных

In [8]:
words_data = data.copy()
words_data['Text'] = words_data['Text'].apply(lambda s: review_to_words(s))
words_data.head(3)

,Sentiment,URL,Text
0,-1,kinopoisk.ru/user/72383/comment/846161/,самых первых титрах появилось абсолютно твердое непоколебимое ощущение смотрю совершено детский фильм духе детей шпионов иначе объяснить наличие начальных титрах техно музыки огромнейшего просто колоссальнейшего наличия компьютерной графики уровня приставок фирмы детству удосужился сыграть видео игру мотивам мультфильма король лев адаптированную специально платформера прекрасно поймет полноты картины хватает мангуста димона который выдал камеру памятное хотя данном фильме фильму подошло сидишь неволей думаешь попал фильм адресованный поколению ясельной группы детского сада сюжет большему с...
1,1,kinopoisk.ru/user/697175/comment/805374/,знаете неожиданно хороший фильм большой любитель фильмов такого жанра очень понравился ожидала придраться сильно хотелось это сделать чему сюжет необычный непривычный оригинальный оторваться просмотра практически невозможно весь фильм проходит движении вечно случается открываются какие тайны всевозможные неожиданные повороты напряженные моменты сказала начнете смотреть фильм остановиться сможете примеру просто смогла оторваться практически прилипла экрану персонажи замечательные помощи актеров которых подобрали ким бейсингер просто шикарна хотя видеть роли жертвы очень непривычно равно здо...
2,1,kinopoisk.ru/user/1335875/comment/2079675/,фильм покажут угадать заочно достаточно посмотреть постер прочесть название однако решил ввязаться просмотр принципе могу сказать жалею времени потраченного это кино это выдающаяся картина увлекательно посмотреть приятных глазу актеров временами смешно это мелодрама элементами комедии любовная комедия комедия девичьим уклоном однако думаю парням посмотреть дебюте вообще главные герои занимаются сексом везде каждом месте словно слетевшие катушек эштон кутчер актер который никак профессионал сформироваться моем сознании часто картины номинируются золотую малину эффект бабочки всякий заставля...


8. Запустите преобразователь текстового признака в «bag of words» с помощью класса CountVectorizer библиотеки sklearn.feature_extraction.text. 

В конструкторе класса укажите параметры: analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 50000, binary = True

Вызовите метод обучения (построения словаря) и преобразования текстов: fit_transform. Результат присвойте новой переменной bagOfWords.

Сохраните порядок слов в словаре (метод .get_feature_names()) в переменную vocabulary.

In [9]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(analyzer='word', tokenizer=None, preprocessor=None,
                                                             stop_words=None, max_features=50000,binary=True)

bagOfWords = vectorizer.fit_transform(words_data['Text'])

vocabulary = vectorizer.get_feature_names_out()

In [10]:
#numpy
vocabulary.shape

(50000,)

In [11]:
#scipy
bagOfWords.shape

(100000, 50000)

In [13]:
bagOfWords

<100000x50000 sparse matrix of type '<class 'numpy.int64'>'
	with 16553366 stored elements in Compressed Sparse Row format>

9. Теперь в наших матрицах объектов-признаков 50 тысяч столбцов и в сумме 100 тысяч строк. Один элемент матрицы хранится в типе double и занимает 8 байт, значит суммарный объем матрицы равен 40 Гб. Проверьте по диспетчеру задач (системный монитор в Linux), сколько занимает памяти ваша программа и напишите в комментариях. Если размеры не согласуются, объясните это в комментариях.

In [14]:
#память: 1475Мб
#предположительно, дело в том, что bagOfWords - разреженная матрица (судя по описанию выше),
#что существенно уменьшает необходимый объём: 16553366 elems * 8байт = 132,426,928байт ~ 129,324Кб ~ 127Мб
#                                             ещё два int на индексы (по крайней мере так оно работало во фрифеме),
#               но не уверен какой инт, но даже если int64, то в сумме не больше 400Мб - сила разреженных матриц :O

10. Разбейте выборку bagOfWords на две части: обучающую (строки до 50000) и проверочную (строки >= 50000). Назовите их bagOfWordsTrain и bagOfWordsTest

In [15]:
bagOfWordsTrain = bagOfWords[:50000]
bagOfWordsTest  = bagOfWords[50000:]

bagOfWordsTrain.shape, bagOfWordsTest.shape

((50000, 50000), (50000, 50000))

11. Сконструируйте машину опорных векторов svm.LinearSVC с параметрами C=1, fit_intercept=False и присвойте ее переменной SVM.

Натренируйте ее на обучающей выборке и с помощью метода .score(x,y) вычислите процент правильных предсказаний для тестовой выборки.

Подберите важность суммы нарушений C, чтобы SVM работала лучше всего

In [17]:
y_train = words_data['Sentiment'][:50000]
y_test  = words_data['Sentiment'][50000:]

y_train.shape, y_test.shape

((50000,), (50000,))

In [19]:
SVM = sklearn.svm.LinearSVC(C=1, fit_intercept=False)

SVM.fit(bagOfWordsTrain, y_train)

acc = SVM.score(bagOfWordsTest, y_test)

acc

C:\Users\melo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.9102

In [20]:
max_acc = 0.92552
max_C = 0.004

In [44]:
current_C = 0.004

SVM = sklearn.svm.LinearSVC(C=current_C, fit_intercept=False)
SVM.fit(bagOfWordsTrain, y_train)
acc = SVM.score(bagOfWordsTest, y_test)

if acc > max_acc:
    max_acc = acc
    max_C = current_C

print(f"{'current acc' :14s} : {acc}; C = {current_C :1.3f}")
print(f"{'max acc' :14s} : {max_acc}; C = {max_C :1.3f}")

C:\Users\melo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


current acc    : 0.92552; C = 0.004
max acc        : 0.92552; C = 0.004


In [45]:
max_acc, max_C

(0.92552, 0.004)

12. Выведите на экран 20 слов, которые больше всего голосуют за позитивность отзыва и 20 — за негативность вместе с соответствующими им коэффициентами натренированной линейной функции:

```python
ind = np.argsort(SVM.coef_[0])
for i in range(20):
    print(SVM.coef_[0][ind[i]], vocabulary[ind[i]])
print("\n")
for i in range(20):
    print(SVM.coef_[0][ind[-i-1]], vocabulary[ind[-i-1]]) 
```

In [49]:
ind = np.argsort(SVM.coef_[0])

for i in range(20):
    print(f'coef = {SVM.coef_[0][ind[i]] :+1.17f}: word = {vocabulary[ind[i]]}')
print("\n")
for i in range(20):
    print(f'coef = {SVM.coef_[0][ind[-i-1]] :+1.17f}: word = {vocabulary[ind[-i-1]]}')

coef = -0.30084529005652910: word = разочарование
coef = -0.26247751868732622: word = увы
coef = -0.25185490422213996: word = скучно
coef = -0.24340186251765006: word = скучный
coef = -0.24111675342255187: word = провал
coef = -0.21801672552472065: word = никакой
coef = -0.19824705435764839: word = отсутствует
coef = -0.19282766701776524: word = единственное
coef = -0.18996132615474329: word = неприятно
coef = -0.18298260455439216: word = надеялся
coef = -0.18095223352008880: word = досмотреть
coef = -0.18042375618869944: word = понимаю
coef = -0.17818186997404645: word = извините
coef = -0.17713719340916112: word = верю
coef = -0.17679696027283126: word = скучен
coef = -0.17621408797125815: word = разочарована
coef = -0.17386414684369833: word = ужасен
coef = -0.17185794268190505: word = неужели
coef = -0.17050417044385618: word = никакого
coef = -0.16809204405915415: word = бред


coef = +0.21368817004206217: word = отличный
coef = +0.18748005087883213: word = великолепно
coef = +0.1

13. Какой коэффициент соответствует словам: 'критика', 'странно', 'понимаю', 'деньги', 'даже' и еще 5 ваших слов по выбору. За что они голосуют: позитивность или негативность отзыва?

In [52]:
additional_words = np.random.choice(vocabulary, 5)
additional_words

array(['снятое', 'поцелуе', 'уродливого', 'противном', 'уничтожены'],
      dtype=object)

In [72]:
check_words = np.concatenate([['критика', 'странно', 'понимаю', 'деньги', 'даже'], additional_words])

for word in check_words:
    
    if word in vocabulary:
        i = np.argwhere(vocabulary == word)[0,0]
        coef = SVM.coef_[0][i]
        
        if coef > 0:
            print(f'слово "{word :10s}" имеет позитивный характер; coef = {coef :+5f}')
        elif coef < 0:
            print(f'слово "{word :10s}" имеет негативный характер; coef = {coef :+5f}')
        else:
            print(f'слово "{word :10s}" нейтрально; coef = {coef :+5f}')
    
    else:
        print(f'слова "{word :10s}" нету')


#word_coef_dict = dict(zip(vocabulary, SVM.coef_[0]))
#  coef = word_coef_dict[word]

слово "критика   " имеет позитивный характер; coef = +0.001636
слово "странно   " имеет позитивный характер; coef = +0.049209
слово "понимаю   " имеет негативный характер; coef = -0.180424
слово "деньги    " имеет негативный характер; coef = -0.080112
слова "даже      " нету
слово "снятое    " имеет негативный характер; coef = -0.007481
слово "поцелуе   " имеет негативный характер; coef = -0.032493
слово "уродливого" имеет негативный характер; coef = -0.017515
слово "противном " имеет позитивный характер; coef = +0.020352
слово "уничтожены" имеет позитивный характер; coef = +0.003265


In [71]:
i = np.argwhere(vocabulary == 'снятое')[0,0]
vocabulary[i], SVM.coef_[0][i]

('снятое', -0.00748105470168903)

14. Вычислите выступы, найдите и выведете на экран отзывы с минимальным и максимальным выступом, величину выступа, величину предсказания и сентиментность отзыва. 

```python
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, data.loc[50000:]['Sentiment'].as_matrix())
ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
print("\n")
i = ind[0] + 50000
print('Величина выступа = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
```

In [82]:
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, y_test.to_numpy())


ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа      = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
print("\n")

i = ind[0] + 50000
print('Величина выступа      = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])

Величина выступа      =  6.426008047201401
Величина предсказания =  6.426008047201401
Сентиментность отзыва =  1


Чудесно! Великолепно! Потрясающе! Ярко! Динамично! Захватывающе! Невероятно! Свежо! Вдохновляюще! Незабываемо! Волшебно! Сказочно! И все это о данной картине... Только из кинотеатра, эмоции одолевают, но все же надо собратся и сказать, почему каждому следует ознакомиться с «Иллюзией обмана»... На что в первую очередь обращает свое внимание зритель? Правильно, на актеров. В этом плане Джесси Айзенберг и Вуди Харрельсон полностью перетягиваю канат на себя. Не отрицаю, у каждого из «великолепной четверки» есть собственная изюминка, но именно образы картежника и гипнотизера запоминаются. Быть может, это из-за их своеобразного соперничества, или же виной этому искрометный юмор, определенная доля наглости и, конечно же, потрясающий талант, но перед этими исполнителями преклоняюсь. Чуть позже в сюжет вплетается и герой Марка Руффало. Он - полная противоположность известных всадни

15. Нормализуйте признаки перед разбиением выборки на Train и Test так, чтобы вместо 0/1 стояли частоты употребления слов, деленные на длину отзыва. Подберите наилучшую константу C и опишите, как изменятся отзывы с минимальным и максимальным выступом? Что лучше: не нормализованные или нормализованные признаки? Почему?

Нужно установить binary = False в CountVectorizer и добавить после fit_transform
```python
bagOfWords = preprocessing.normalize(bagOfWords, norm='l1')
```

In [84]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(analyzer='word', tokenizer=None, preprocessor=None,
                                                             stop_words=None, max_features=50000,binary=False)

bagOfWords = vectorizer.fit_transform(words_data['Text'])
bagOfWords = sklearn.preprocessing.normalize(bagOfWords, norm='l1')
vocabulary = vectorizer.get_feature_names_out()

In [85]:
bagOfWordsTrain = bagOfWords[:50000]
bagOfWordsTest  = bagOfWords[50000:]

bagOfWordsTrain.shape, bagOfWordsTest.shape

((50000, 50000), (50000, 50000))

In [86]:
max_acc = 0.92754
max_C = 120

In [109]:
current_C = 50

SVM = sklearn.svm.LinearSVC(C=current_C, fit_intercept=False)
SVM.fit(bagOfWordsTrain, y_train)
acc = SVM.score(bagOfWordsTest, y_test)

if acc > max_acc:
    max_acc = acc
    max_C = current_C

print(f"{'current acc' :14s} : {acc}; C = {current_C :1.3f}")
print(f"{'max acc' :14s} : {max_acc}; C = {max_C :1.3f}")

C:\Users\melo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


current acc    : 0.925; C = 50.000
max acc        : 0.92754; C = 120.000


In [110]:
max_acc, max_C

(0.92754, 120)

In [111]:
#старая max_acc=0.92552
#новая  max_acc=0.92754

In [114]:
prediction = SVM.decision_function(bagOfWordsTest)
margins = np.multiply(prediction, y_test.to_numpy())


ind = np.argsort(margins)
i = ind[-1] + 50000
print('Величина выступа      = ', margins[ind[-1]])
print('Величина предсказания = ', prediction[ind[-1]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
print("\n")

i = ind[0] + 50000
print('Величина выступа      = ', margins[ind[0]])
print('Величина предсказания = ', prediction[ind[0]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])
print("\n")

i = ind[1] + 50000
print('Величина выступа      = ', margins[ind[1]])
print('Величина предсказания = ', prediction[ind[1]])
print('Сентиментность отзыва = ', data.loc[i,'Sentiment'])
print('\n')
print(data.loc[i,'Text'])

Величина выступа      =  6.7174269181550885
Величина предсказания =  6.7174269181550885
Сентиментность отзыва =  1


Действительно, интересный молодежный фильм, немного наивный, но приятный для просмотра, ну, и грустный местами. Он на все 100% дает то, что можно ожидать от таких фильмов, и может даже немного больше! .


Величина выступа      =  -3.412507439949425
Величина предсказания =  -3.412507439949425
Сентиментность отзыва =  1


«Придурки» продолжают свое победное шествие по большим экранах. Их игрища конечно на любителя, но их второе пришествие меня разочаровала по сравнению с первой частью. Иногда смешно, иногда очень смешно, то чаще противно, тупо и ужасно и.... Вывод - поржать можно, но от употребления пищи при этом следует отказаться, а то постигнет разочарование...


Величина выступа      =  -3.412507439949425
Величина предсказания =  -3.412507439949425
Сентиментность отзыва =  1


Ну что я могу добавить. Для проведения времени, фильм можно просмотреть один раз и забыть про

In [115]:
#старый выступ+: 6.426008047201401
#новый  выступ+: 6.7174269181550885
#
#старый выступ-: -2.9774941897109954
#новый  выступ-: -3.412507439949425
#
#
#
#нормализация повлекла обратный эффект при подборе C: до этого C бралось как можно меньше,
#  а после - точчность увеличивалась при бОльших C, но бОльшие С программа считала чуть дольше
#
#score при нормализации увеличился на 0.02
#
#все три крайние комментария в сравнении с ненормализованными - очень даже коротки,
#  похоже на побочный эффект
#
#в случае + выступ сильно не изменился;
#в случае - выступ стал меньше => модель была более уверена в своём неправильном выборе
#
#касательно содержания комментариев:
#  в обоих случаях положительные комментарии довольно показательны - модель справилась
#  в случае негативных:
#    ненормализованная - взяла комментарий, где много критики. автор счёл чрезвычайно важным описать свои чувства
#     касательно книжной индустрии, но фильм в итоге скорее хвалил, чем ругал (вроде. не обзорно я это читать не намерен).
#     так что при таком негативном характере текста не удивительно, что сеть сочла его негативным  - прощаю ошибку
#   
#   нормализованная - искренне не могу понять, как при таком отзыве оценка стоит положительная.
#    решил прочесть второй по 'неправильности' - история та же; как можно писать такое и ставить +1
#    - самые настоящие шумовые данные

16. Объясните в комментариях, почему найденный отзыв с очень отрицательным выступом имеет такой маленький выступ, а другой найденный — такой большой.

Для объяснения отступов вам возможно пригодится код, который печатает коэффициенты w<sub>i</sub> линейной функции, значения соответствующих признаков и соответствующие слова. И все это делается в порядке убывания модуля произведения w<sub>i</sub>x<sub>i</sub>:
```python
k = ind[0]
ind2 = np.argsort(np.abs(np.multiply(SVM.coef_[0], bagOfWordsTest[k].todense()))).reshape(-1,1)
s = 0
for i in range((bagOfWordsTest[k]!=0).sum()):
    ii = ind2[-i-1].item()
    print('wi='+str(SVM.coef_[0][ii]), 'xi='+str(bagOfWordsTest[k,ii]),  vocabulary[ii])
    s += SVM.coef_[0][ii]*bagOfWordsTest[k,ii]
print('Проверка:',s) 
```

In [116]:
k = ind[0]
ind2 = np.argsort(np.abs(np.multiply(SVM.coef_[0], bagOfWordsTest[k].todense()))).reshape(-1,1)

s = 0
for i in range((bagOfWordsTest[k]!=0).sum()):
    ii = ind2[-i-1].item()
    print('wi='+str(SVM.coef_[0][ii]), 'xi='+str(bagOfWordsTest[k,ii]),  vocabulary[ii], SVM.coef_[0][ii]*bagOfWordsTest[k,ii])
    s += SVM.coef_[0][ii]*bagOfWordsTest[k,ii]

print('Проверка:',s)

wi=-36.484099590078685 xi=0.03571428571428571 разочарование -1.3030035567885243
wi=-10.004466015993556 xi=0.07142857142857142 смешно -0.7146047154281111
wi=-16.88463144706584 xi=0.03571428571428571 ужасно -0.6030225516809228
wi=-14.062973344465997 xi=0.03571428571428571 противно -0.5022490480166427
wi=-12.125548699511569 xi=0.03571428571428571 тупо -0.4330553106968417
wi=-8.893540863106253 xi=0.03571428571428571 вывод -0.31762645939665185
wi=8.65304273445289 xi=0.03571428571428571 очень 0.30903724051617465
wi=-5.694801053707678 xi=0.03571428571428571 разочаровала -0.20338575191813135
wi=-5.17332917969604 xi=0.03571428571428571 продолжают -0.1847617564177157
wi=4.840580590619833 xi=0.03571428571428571 отказаться 0.17287787823642262
wi=4.3209784341151325 xi=0.03571428571428571 чаще 0.15432065836125472
wi=4.102086708294431 xi=0.03571428571428571 сравнению 0.1465030967248011
wi=-3.190631832488448 xi=0.03571428571428571 экранах -0.11395113687458742
wi=3.132696262871373 xi=0.0357142857142857

In [118]:
#предположительно, почему большой по модулю отрицательный выступ
# 'смешно' имеет резко отрицательный характер (-10), хотя может быть использовано как в отрицательном,
#   так и в положительном контексте
# 'очень', 'чаще', 'сравнению', 'экранах', 'свое', ...  - тоже довольно нейтральные слова, которые контринтуитивно
#   имеют немаленький модуль у весов
#
#Важнее:
#как и говорилось в коментарии в 15), такие слова, как 'разочарование', 'ужасно',
#  'противно', 'тупо', 'разочаровала', 'придурки', - несомненно имеют чрезвычайно отрицательную коннотацию.
#  что они забыли в положительном отзыве, я не знаю.

In [117]:
k = ind[-1]
ind2 = np.argsort(np.abs(np.multiply(SVM.coef_[0], bagOfWordsTest[k].todense()))).reshape(-1,1)
s = 0
for i in range((bagOfWordsTest[k]!=0).sum()):
    ii = ind2[-i-1].item()
    print('wi='+str(SVM.coef_[0][ii]), 'xi='+str(bagOfWordsTest[k,ii]),  vocabulary[ii], SVM.coef_[0][ii]*bagOfWordsTest[k,ii])
    s += SVM.coef_[0][ii]*bagOfWordsTest[k,ii]
print('Проверка:',s)

wi=14.0658617156716 xi=0.13333333333333333 немного 1.8754482287562133
wi=13.336904849880572 xi=0.06666666666666667 интересный 0.8891269899920381
wi=11.320993149128148 xi=0.06666666666666667 дает 0.7547328766085432
wi=10.16532645081 xi=0.06666666666666667 приятный 0.6776884300539999
wi=7.93583154091352 xi=0.06666666666666667 ожидать 0.5290554360609013
wi=7.390373087261641 xi=0.06666666666666667 наивный 0.49269153915077607
wi=4.505515805355776 xi=0.06666666666666667 местами 0.3003677203570517
wi=3.6909527523505927 xi=0.06666666666666667 грустный 0.24606351682337285
wi=3.6739800631019692 xi=0.06666666666666667 просмотра 0.24493200420679795
wi=3.5605990984371974 xi=0.06666666666666667 таких 0.23737327322914648
wi=1.9754962650895262 xi=0.06666666666666667 фильмов 0.13169975100596842
wi=1.877963837598475 xi=0.06666666666666667 молодежный 0.12519758917323168
wi=1.6135901446059417 xi=0.06666666666666667 фильм 0.10757267630706278
wi=1.5821532964497507 xi=0.06666666666666667 действительно 0.1054

In [ ]:
#верю, что такие слова используются в положительном отзыве
#
#вывод (основанный исключительно на 1+2 экземплярах с отрицательным выступом...):
# виноваты шумовые объекты